<a href="https://colab.research.google.com/github/fclubb/EarthSurfaceProcesses/blob/master/Week2_HillslopeEvolution/2DHillslope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hillslope evolution in 2D using Landlab

_This notebook is adapted from Landscape Evolution Modeling with CHILD by Gregory Tucker and Stephen Lancaster, and from the Landlab notebook by Nicole Gasparini at Tulane University._

Last year in Mountain Landscapes, you simulated how a simple hillslope would evolve via both linear and non-linear hillslope diffusion. You did this in 1D - you had one cross section of a hillslope. You can see how you already managed to explore a lot of interesting things about hillslope morphology from this 1D approach!

But what if we're interested in how hillslopes interact with each other or with the channel network, or how morphology varies spatially? To look at that, we need to move beyond one to two dimensions - where we are simulating hillslopes in both an X and a Y dimension. To do that, we are going to use the _Landlab_ numerical model. Landlab is a piece of software that has been developed to model landscapes in Python. It is very flexible: we can choose to simulate lots of different processes all within one piece of software. You will be using Landlab later in the module to simulate channel processes as well. 

You also have the option of using Landlab to complete your independent project report (more on that later). 

## Quick reminder:

**To run a code block, click in a cell, hold down shift, and press enter.** An asterisk in square brackets `In [*]:` will appear while the code is being executed, and this will change to a number `[1]:` when the code is finished. *The order in which you execute the code blocks matters, they must be run in sequence.* 

### Working through the practical

To work through this notebook, firstly:
1. **COPY THE NOTEBOOK TO YOUR GOOGLE DRIVE** using the "Copy to Drive" button at the top of the page. 
2. Read through the instructions and execute each code block cell by clicking `Shift and Enter` to see what it does.
3. Do the exercises set out throughout notebook.
4. Save the figures and keep them for the next session.



<hr>
For tutorials on learning Landlab, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a>
<hr>


**Linear diffusion**

Just like last year, we will start by exploring how hillslope morphology evolves when we use the linear diffusion law. You should remember that this takes the form: 

$$
\begin{equation}
q_s = DS \qquad \textrm{where} \qquad S = {{dz}\over{dx}}
\end{equation}
$$
Note that $z$ denotes elevation at the surface and $x$ distance along the hillslope, so the slope is related to the hillslope angle. 
 
Changes in elevation, or erosion, are calculated from conservation of mass:
\begin{equation}
\frac{dz}{dt} = U-\nabla q_s
\end{equation}
where $U$ is the rock uplift rate, with dimensions LT$^{-1}$.

**How will we explore this with Landlab?**

We will use the Landlab component *LinearDiffuser*, which implements the equations above, to explore how hillslopes evolve when linear diffusion describes hillslope sediment transport. We will explore both steady state, here defined as erosion rate equal to rock uplift rate, and also how a landscape gets to steady state.

The first example illustrates how to set-up the model and evolve a hillslope to steady state, along with how to plot some variables of interest. We assume that you have knowledge of how to derive the steady-state form of a uniformly uplifting, steady-state, diffusive hillslope. For more information on hillslope sediment transport laws, this paper is a great overview:

------------

Roering, J. (2008) How well can hillslope evolution models “explain” topography? Simulating soil transport and production with high-resolution topographic data. *Geological Society of America Bulletin.*

-----------

Based on the first example, you are asked to first think about what will happen as you change a parameter, and then you explore this numerically by changing the code.

Start at the top by reading each block of text and sequentially running each code block (shift - enter OR got to the _Cell_ pulldown menu at the top and choose _Run Cells_). 

Remember that you can always go to the _Kernel_ pulldown menu at the top and choose _Restart & Clear Output_ or _Restart & Run All_ if you change things and want to start afresh. If you just change one code block and rerun only that code block, only the parts of the code in that code block will be updated. (E.g. if you change parameters but don't reset the code blocks that initialize run time or topography, then these values will not be reset.) 

## Installing Landlab
Google Colab doesn't have landlab installed automatically, so we will have to run some code to install it. To do that, run the below code block. This will output a lot of text which you can ignore.
**You should only need to do this once when you start the notebook up - if you restart the runtime, then there is no need to re-run this code block**

In [ ]:
!pip install numpy==1.20.1 &> /dev/null
!pip install landlab &> /dev/null

**Now on to the code example**

Import statements. You should not need to edit this.


In [ ]:
# below is to make plots show up in the notebook
%matplotlib inline

**IF YOU GET AN ERROR HERE, click on "Runtime" in the top menu, and then "Restart runtime". Then try running this cell again.**

In [ ]:
# Code Block 1

import numpy as np
from matplotlib.pyplot import figure, legend, plot, show, title, xlabel, ylabel, ylim

from landlab.plot.imshow import imshow_grid

We will create a grid with 50 rows and 50 columns, and dx (the model resolution, or size of each pixel) is 5 m. The initial elevation is 0 at all nodes.

We set-up boundary conditions so that material can leave the hillslope at the two short ends.

In [ ]:
# Code Block 2

# setup grid
from landlab import RasterModelGrid

mg = RasterModelGrid((50, 50), 5.0)
z_vals = mg.add_zeros("topographic__elevation", at="node")

# initialize some values for plotting
ycoord_rast = mg.node_vector_to_raster(mg.node_y)
ys_grid = ycoord_rast[:, 2]

# set boundary condition.
mg.set_closed_boundaries_at_grid_edges(True, False, True, False)

Now we import and initialize the *LinearDiffuser* component. 

IMPORTANT: In this block we set the "D" parameter which is the diffusivity parameter in the linear hillslope equation.

In [ ]:
# Code Block 3

from landlab.components import LinearDiffuser

D = 0.01  # initial value of 0.01 m^2/yr
lin_diffuse = LinearDiffuser(mg, linear_diffusivity=D)

We now initialize a few more parameters.

In [ ]:
# Code Block 4

# Uniform rate of rock uplift
uplift_rate = 0.0001  # meters/year, originally set to 0.0001

# Total time in years that the model will run for.
runtime = 1000000  # years, originally set to 1,000,000

# Stability criteria for timestep dt.  Coefficient can be changed
# depending on our tolerance for stability vs tolerance for run time.
dt = 0.5 * mg.dx * mg.dx / D

# nt is number of time steps
nt = int(runtime // dt)

# Below is to keep track of time for labeling plots
time_counter = 0

# length of uplift over a single time step, meters
uplift_per_step = uplift_rate * dt

Now we figure out the analytical solution for the elevation of the steady-state profile. 

This step basically tells us what topography we would expect our hillslope to have just based on the equation, without doing any numerical modelling. It predicts the STEADY STATE hillslope morphology: once erosion of the hillslope has adjusted to account for the rate of uplift, the morphology of the hillslope shouldn't change through time.

In [ ]:
# Code Block 5

ys = np.arange(mg.number_of_node_rows * mg.dx - mg.dx)

# location of divide or ridge crest -> middle of grid
# based on boundary conds.
divide_loc = (mg.number_of_node_rows * mg.dx - mg.dx) / 2

# half-width of the ridge
half_width = (mg.number_of_node_rows * mg.dx - mg.dx) / 2

# analytical solution for elevation under linear diffusion at steady state
zs = (uplift_rate / (2 * D)) * (np.power(half_width, 2) - np.power(ys - divide_loc, 2))

Before we evolve the landscape, let's look at the initial topography. The first figure shows a map view of our hillslope coloured by elevation, while the second figure shows you a cross section from north to south across the model domain (The red line, just verifying that it is flat with zero elevation).

In [ ]:
# Code Block 6

figure(1)
imshow_grid(mg, "topographic__elevation")
title("initial topography")
figure(2)
elev_rast = mg.node_vector_to_raster(mg.at_node["topographic__elevation"])
plot(ys_grid, elev_rast[:, 2], "r-", label="model")
plot(ys, zs, "k--", label="analytical solution")
ylim((-5, 50))  # may want to change upper limit if D changes
xlabel("horizontal distance (m)")
ylabel("vertical distance (m)")
legend(loc="lower center")
title("initial topographic cross section")

Now we are ready to evolve the landscape and compare it to the steady state solution.

Below is the time loop that does all the calculations. 

In [ ]:
# Code Block 7

for i in range(nt):
    mg["node"]["topographic__elevation"][mg.core_nodes] += uplift_per_step
    lin_diffuse.run_one_step(dt)
    time_counter += dt

    # All landscape evolution is the first two lines of loop.
    # Below is simply for plotting the topography halfway through the run
    if i == int(nt // 2):
        figure(1)
        imshow_grid(mg, "topographic__elevation")
        title("topography at time %s, with D = %s" % (time_counter, D))
        figure(2)
        elev_rast = mg.node_vector_to_raster(mg.at_node["topographic__elevation"])
        plot(ys_grid, elev_rast[:, 2], "k-", label="model")
        plot(ys, zs, "g--", label="analytical solution - SS")
        plot(ys, zs * 0.75, "b--", label="75% of analytical solution")
        plot(ys, zs * 0.5, "r--", label="50% of analytical solution")
        xlabel("horizontal distance (m)")
        ylabel("vertical distance (m)")
        legend(loc="lower center")
        title("topographic__elevation at time %s, with D = %s" % (time_counter, D))

Now we plot the final cross-section.

In [ ]:
# Code Block 8

elev_rast = mg.node_vector_to_raster(mg.at_node["topographic__elevation"])
plot(ys_grid, elev_rast[:, 2], "k-", label="model")
plot(ys, zs, "g--", label="analytical solution - SS")
plot(ys, zs * 0.75, "b--", label="75% of analytical solution")
plot(ys, zs * 0.5, "r--", label="50% of analytical solution")
xlabel("horizontal distance (m)")
ylabel("vertical distance (m)")
legend(loc="lower center")
title("topographic cross section at time %s, with D = %s" % (time_counter, D))

Now we plot the steepest slope in the downward direction across the landscape.

(To calculate the steepest slope at a location, we need to route flow across the landscape.)

In [ ]:
# Code Block 9

from landlab.components import FlowAccumulator

fr = FlowAccumulator(mg)  # intializing flow routing
fr.run_one_step()
plot(
    mg.node_y[mg.core_nodes],
    mg.at_node["topographic__steepest_slope"][mg.core_nodes],
    "k-",
)
xlabel("horizontal distance (m)")
ylabel("topographic slope (m/m)")
title("slope of the hillslope at time %s, with D = %s" % (time_counter, D))

Has the landscape reached steady state yet? How do you know?







Answer: Not quite, but it is getting close. Go back and rerun Code Blocks 7, 8 and 9 (time loop and plotting). (Remember you can rerun a cell with shift-return, or from the cell pull-down menu.) Has it reached steady state yet?  

**Exercises:**
1. In the example illustrated here: $D$ = 0.01 m$^2$yr$^{-1}$, and $U$ = 0.0001 m yr$^{-1}$. Restart everything, and use the model to determine how long it takes for the landscape to go from a flat to reach 50%, 75% and 100% of its steady-state morphology. Does the landscape approach steady state linearly in time? (You can run the time loop (Code Block 7) multiple times without running other code blocks again to continually evolve the landscape. You will initially want to rerun all the code blocks and change the value of **run_time** (Code Block 4). Determining the correct value of **run_time** to use will take some iteration.
2. What do you think will happen when you increase $D$ (Code Block 3) by a factor of 10? Will the time to steady state differ? If yes, how? Will the topography be different? If yes, how and why? What does it mean physically, about processes, if $D$ increases? Answer these questions before running any code. 
3. Now set $D$ = 0.1 m$^2$yr$^{-1}$ and rerun landscape evolution from an initial flat. Illustrate the final steady state topography and record the time to steady state. Discuss how the landscape differs from the results in question 1. Discuss how the results are similar to or different from your intuition. It is OK if your intuition was wrong! 
4. What do you think will happen when you increase **uplift_rate** (Code Block 4) by a factor of 10? Will the time to steady state differ? If yes, how? Will the topography be different? If yes, how and why? Answer these questions first, and then rerun the code with **uplift_rate** = 0.001 m yr$^{-1}$. (Make sure you change $D$ - Code Block 3 - back to the original value of 0.01 m$^2$yr$^{-1}$ and restart from a flat surface.) Illustrate the final steady state topography. Discuss how these results differ from the results in question 1 and how the results match (or do not) your intuition. It is OK if your intuition was wrong.

These exercises will not be assessed, but understanding how numerical models work and the concepts of steady state plus hillslope diffusion are really important for the rest of the module. You can also use some of the material from these notebooks for your independent project.


--------------
## Some extra code that can be helpful for projects

### Saving the model topography as a DEM

If you want to create a DEM of the topography that you have made, you can save it as an ASCII file (the file should end in .asc). You can then load the topography into a GIS to do extra analyses if you like. To do that, specify the filename you want to use here and then run the cell block. You will then need to download the DEM from Google Colab

In [ ]:
# Code Block 10 

from landlab.io import write_esri_ascii

write_file_name = '2D_Hillslope_topography_linear.asc'
# Below is writing elevation data in the ESRI ascii format so that it can
# easily be read into Arc GIS or back into Landlab.
write_esri_ascii(write_file_name, mg, 'topographic__elevation')

# write the steepest slope to a DEM
write_slope_name = '2D_Hillslope_steepest_slope_linear.asc'
write_esri_ascii(write_slope_name, mg, 'topographic__steepest_slope')

-----------------------
### Running the model with non-linear diffusion

To run the model using the non-linear diffusion model, you can use the code below.

We will create a grid with 50 rows and 50 columns, and dx (the model resolution, or size of each pixel) is 5 m. The initial elevation is 0 at all nodes.

We set-up boundary conditions so that material can leave the hillslope at the two short ends.

In [ ]:
# Code Block 11

# setup grid
from landlab import RasterModelGrid

mg = RasterModelGrid((50, 50), 5.0)
z_vals = mg.add_zeros("topographic__elevation", at="node")

# initialize some values for plotting
ycoord_rast = mg.node_vector_to_raster(mg.node_y)
ys_grid = ycoord_rast[:, 2]

# set boundary condition.
mg.set_closed_boundaries_at_grid_edges(True, False, True, False)

Now we import and initialize the *NonLinearDiffuser* component. 

In [ ]:
# Code Block 12

from landlab.components import TaylorNonLinearDiffuser

S_C = 0.1 # set the critical slope value here
D = 0.01  # initial value of 0.01 m^2/yr

cubicflux = TaylorNonLinearDiffuser(mg, slope_crit=S_C, linear_diffusivity=D, dynamic_dt=True)

We now initialize a few more parameters.

In [ ]:
# Code Block 13

# Uniform rate of rock uplift
uplift_rate = 0.0001  # meters/year, originally set to 0.0001

# Total time in years that the model will run for.
runtime = 1000000  # years, originally set to 1,000,000

# Stability criteria for timestep dt.  Coefficient can be changed
# depending on our tolerance for stability vs tolerance for run time.
dt = 0.5 * mg.dx * mg.dx / D

# nt is number of time steps
nt = int(runtime // dt)

# Below is to keep track of time for labeling plots
time_counter = 0

# length of uplift over a single time step, meters
uplift_per_step = uplift_rate * dt

Before we evolve the landscape, let's look at the initial topography. (This is just verifying that it is flat with zero elevation.)

In [ ]:
# Code Block 14

figure(1)
imshow_grid(mg, "topographic__elevation")
title("initial topography")
elev_rast = mg.node_vector_to_raster(mg.at_node["topographic__elevation"])

Now we are ready to evolve the landscape and compare it to the steady state solution.

Below is the time loop that does all the calculations. 

In [ ]:
# Code Block 15

for i in range(nt):
    mg["node"]["topographic__elevation"][mg.core_nodes] += uplift_per_step
    cubicflux.run_one_step(dt)
    time_counter += dt

    # All landscape evolution is the first two lines of loop.
    # Below is simply for plotting the topography halfway through the run
    if i == int(nt // 2):
        figure(1)
        imshow_grid(mg, "topographic__elevation")
        title("topography at time %s, with D = %s" % (time_counter, D))
        figure(2)
        elev_rast = mg.node_vector_to_raster(mg.at_node["topographic__elevation"])
        plot(ys_grid, elev_rast[:, 2], "k-", label="Model elevation")
        xlabel("horizontal distance (m)")
        ylabel("vertical distance (m)")
        legend(loc="lower center")
        title("topographic__elevation at time %s, with D = %s" % (time_counter, D))

Now we plot the final cross-section.

In [ ]:
# Code Block 16

elev_rast = mg.node_vector_to_raster(mg.at_node["topographic__elevation"])
plot(ys_grid, elev_rast[:, 2], "k-", label="Model elevation")
xlabel("horizontal distance (m)")
ylabel("vertical distance (m)")
legend(loc="lower center")
title("topographic cross section at time %s, with D = %s" % (time_counter, D))

Now we plot the steepest slope in the downward direction across the landscape.

(To calculate the steepest slope at a location, we need to route flow across the landscape.)

In [ ]:
# Code Block 17

from landlab.components import FlowAccumulator

fr = FlowAccumulator(mg)  # intializing flow routing
fr.run_one_step()
plot(
    mg.node_y[mg.core_nodes],
    mg.at_node["topographic__steepest_slope"][mg.core_nodes],
    "k-",
)
xlabel("horizontal distance (m)")
ylabel("topographic slope (m/m)")
title("slope of the hillslope at time %s, with D = %s" % (time_counter, D))

Now save the elevation and slope as DEMs

In [ ]:
# Code Block 10 

from landlab.io import write_esri_ascii

write_file_name = '2D_Hillslope_topography_nonlinear.asc'
# Below is writing elevation data in the ESRI ascii format so that it can
# easily be read into Arc GIS or back into Landlab.
write_esri_ascii(write_file_name, mg, 'topographic__elevation')

# write the steepest slope to a DEM
write_slope_name = '2D_Hillslope_steepest_slope_nonlinear.asc'
write_esri_ascii(write_slope_name, mg, 'topographic__steepest_slope')